## Mapping YCOM Values in US counties

In [21]:
from bokeh.plotting import figure, show, output_file, ColumnDataSource
import pandas as pd
from collections import OrderedDict
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import Category20

In [22]:
from bokeh.sampledata import us_counties, unemployment

In [23]:
#read in Climate Opinions Data with fips codes
YCOM2018 = pd.read_csv('yale_data_FIPS.csv', encoding='latin-1' )


In [24]:
# make column with fips code as tuple
YCOM2018['FIPS'] = list(zip(YCOM2018['state_code'], YCOM2018['county_code']))

In [25]:
# for first samp plot, subset dataframe to two cols: fips conde and 'happening' column
df = YCOM2018[['happening', 'FIPS']]
df.head()


,happening,FIPS
0,59.476,"(1, 1)"
1,60.062,"(1, 3)"
2,67.862,"(1, 5)"
3,58.232,"(1, 7)"
4,52.956,"(1, 9)"


In [26]:
#convert to dictionary where fips code is key, answer percenage is value
fips_dict = dict(zip(df['FIPS'], df['happening']))



In [27]:
# trying to adapt sample code
state = "tx"
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")
# separating lng data for county shape
county_xs=[
    us_counties.data[code]['lons'] for code in us_counties.data
    if us_counties.data[code]['state'] == state
]

#separating lat data for county shapes 
county_ys=[
    us_counties.data[code]['lats'] for code in us_counties.data
    if us_counties.data[code]['state'] == state
]


colors = Category20[10]


In [30]:
#county_colors = []
county_names = []
county_rates = []

for county_id in us_counties:
    if us_counties[county_id]["state"] in EXCLUDED:
        continue
    try:
        rate = df[county_id]
        idx = int((rate + 10) / 10) * 10
        county_colors.append(colors[idx])
        county_names.append(us_counties.data[county_id]['name'])
        county_rates.append(rate)
    except KeyError:
        county_colors.append("black")    
    



TypeError: 'module' object is not iterable

In [ ]:
source = ColumnDataSource(
    data = dict(
        x=county_xs,
        y=county_ys,
        color=county_colors,
        name=county_names,
        rate=county_rates,
    )
)


output_file("example_plot.html", title="state-level example")

TOOLS="pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="happening", tools=TOOLS)

p.patches('x', 'y',
    fill_color='color', fill_alpha=0.7,
    line_color="white", line_width=0.5,
    source=source)

hover = p.select(dict(type=HoverTool))
hover.point_policy = "follow_mouse"
hover.tooltips = OrderedDict([
    ("Name", "@name"),
    ("CC is happening rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
])

show(p)